In [6]:
import scipy

from scipy.io import savemat

import scipy.io as sio
from numpy.core.records import fromarrays
import numpy as np
from PIL import Image
import pandas as pd
import requests

In [2]:
df = pd.read_csv('../Batch_4774446_batch_results_dummy_0_bbox_2022_07_05_s3_v1.csv')

In [5]:
df['Input.image_url'][0]

'https://robindummy.s3.us-west-2.amazonaws.com/dummy_0_detection_with_gt_2022_07_05_s3/2009_002779_shape.jpg'

In [7]:
img_url = df['Input.image_url'][0]

In [8]:
img = Image.open(requests.get(img_url, stream=True).raw)

In [4]:
['class', 'truncated', 'occluded', 'difficult', 'bbox', ]

['class', 'truncated', 'occluded', 'difficult', 'bbox']

In [5]:
import json

In [6]:
subset_json = json.load(open('subset_test/subset-2.json', 'r'))

In [7]:
subset_json.keys()

dict_keys(['images', 'categories', 'annotations'])

In [8]:
catid2cat = {
    item['id']: item['name'] for item in subset_json['categories']
}

In [9]:
def xywh_to_xyxy(xywh):
    """Convert [x1 y1 w h] box format to [x1 y1 x2 y2] format."""
    if isinstance(xywh, (list, tuple)):
        # Single box given as a list of coordinates
        assert len(xywh) == 4
        x1, y1 = xywh[0], xywh[1]
        x2 = x1 + np.maximum(0., xywh[2] - 1.)
        y2 = y1 + np.maximum(0., xywh[3] - 1.)
        return (x1, y1, x2, y2)
    elif isinstance(xywh, np.ndarray):
        # Multiple boxes given as a 2D ndarray
        return np.hstack(
            (xywh[:, 0:2], xywh[:, 0:2] + np.maximum(0, xywh[:, 2:4] - 1))
        )
    else:
        raise TypeError('Argument xywh must be a list, tuple, or numpy array.')


In [10]:
name2record = {}
for item in subset_json['images']:
    imgid = item['id']
    file_name = item['file_name']
    
    anno_class, anno_truncated, anno_occluded, anno_difficult, anno_bbox = [], [], [], [], []
    
    for anno in subset_json['annotations']:
        if anno['image_id'] == imgid:
            anno_class.append(catid2cat[anno['category_id']])
            anno_truncated.append(anno['metadata']['is_truncated'])
            anno_occluded.append(anno['metadata']['is_occluded'])
            anno_difficult.append(anno['metadata']['is_difficult'])
            bbox = list(map(int, anno['bbox']))
            bbox = list(map(int, xywh_to_xyxy(bbox)))
            anno_bbox.append(np.array(bbox))
            
    obj = {
        'record': {
            'filename': file_name,
            'objects': 
                fromarrays(np.array([
                    anno_class, anno_truncated, anno_occluded, anno_difficult, anno_bbox
                ]), names=['class', 'truncated', 'occluded', 'difficult', 'bbox', ])
        }
    }
    
    name2record.update({
        file_name[:-4] + '.mat': obj
    })

In [11]:
import os.path as osp

In [12]:
for k, v in name2record.items():
    sio.savemat(osp.join('subset_test/subset/Annotations/', k), v)